## PLANTEAMIENTO DEL PROBLEMA
1. Somos parte de una empresa llamada SECOND CARS y vendemos coches usados.
2. Durante un estudio reciente de la empresa, se observó que los vendedores no ponen el precio correctamente a los coches   'Chevrolet'. Algunos de los coches tienen un precio demasiado alto y otros tienen un precio muy bajo, lo que ha causado pérdidas sustanciales en el trimestre anterior.
3. Para remediar esto, la dirección nos ha encargado crear un modelo que prediga de manera eficiente el precio de los automóviles con un error mínimo. De modo que sea útil para los vendedores al fijar correctamente el precio del automóvil.

## PROCESO DE SOLUCIÓN

###  1. Carga de datos

In [ ]:
import numpy as np 
import pandas as pd 

import warnings
warnings.filterwarnings("ignore")

# Carga los datos
df= pd.read_csv('car_price_data.csv')

In [ ]:
# Entiende las dimensiones
df.shape

In [ ]:
# Revisa las 100 primeras filas
df.head()

In [ ]:
# Revisa las estadísticas
df.describe()

###  2. Visualización de datos

In [ ]:
# Plots univariantes: box and whisker
df.plot(kind='box', subplots=True, layout=(2,4), sharex=False, sharey=False)

In [ ]:
# Plots univariantes: histogramas
df.hist()

In [ ]:
from pandas.plotting import scatter_matrix

# Plots multivariantes: scattered_matrix
scatter_matrix(df)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Verifica la correlación
f, ax = plt.subplots(figsize=(10, 8))
corr = df.corr()
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)

In [ ]:
corr.style.background_gradient(cmap='coolwarm', axis=None)

###  3. Preparación de datos

In [ ]:
# Limpia los datos
df.isnull().sum()

In [ ]:
# Completa datos

In [ ]:
# Separa las features del target
target = df[['price']]
X = df[df.columns.drop('price')]

In [ ]:
target.head()

In [ ]:
X.head()

In [ ]:
# Si consideras que hay demasiadas variables aplica PCA

In [ ]:
# Separa variables categóricas de numéricas
num_columns = X.select_dtypes(exclude='object').columns
num_columns=num_columns.drop(pd.Index(['year']))
cat_columns = X.select_dtypes(include='object').columns
cat_columns=cat_columns.append(pd.Index(['year']))

In [ ]:
num_columns, cat_columns

In [ ]:
unordered_columns =['transmission', 'fuel_type']
ordered_columns = ['year', 'model']

In [ ]:
from sklearn.preprocessing import OneHotEncoder 

# Procesa las variables categóricas:One Hot Encoding y OrdinalEncoder
oh_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
oh_cols = pd.DataFrame(oh_encoder.fit_transform(df[unordered_columns]))
oh_cols

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ord_encoder = OrdinalEncoder()
ord_cols = pd.DataFrame(ord_encoder.fit_transform(df[ordered_columns]))
ord_cols

In [ ]:
from sklearn.preprocessing import StandardScaler

# Procesa las variables numéricas: StandardScaler
df_num=df[num_columns]
sc = StandardScaler()
sc_cols = pd.DataFrame(sc.fit_transform(df[num_columns]),columns=num_columns)
sc_cols

In [ ]:
# Genera el dataframe global
X = pd.concat([oh_cols, ord_cols, sc_cols], axis = 1)
X.head()

### 4. Selecciona un modelo

In [ ]:
# Define una lista de candidatos a probar para el problema
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor 
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet

In [ ]:
# Selecciona un algoritmo
model = LinearRegression()

In [ ]:
# Define las métricas

In [ ]:
from sklearn.model_selection import cross_validate,ShuffleSplit

# Genera un análisis CV
cv = ShuffleSplit(n_splits=30, test_size=0.2)

metrics = ['neg_mean_absolute_error', 'r2']
cv_results = cross_validate(model, X, target, cv=cv, scoring=metrics, return_train_score=True)
cv_results = pd.DataFrame(cv_results)
cv_results

In [ ]:
# Verifica los errores de training y testing, así como el overfitting underfitting
import matplotlib.pyplot as plt

scores = pd.DataFrame()
scores[["train error", "test error"]] = cv_results[["train_r2", "test_r2"]]

scores.plot.hist(bins=50, edgecolor="black")
plt.xlabel("r2")
_ = plt.title("Distribución de errores de training y prueba con CV")

#### Otra manera

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=1)

In [ ]:
from sklearn.metrics import mean_squared_error

### Function to test various models
def score_dataset(X_train,X_valid,y_train,y_valid, input_model):
    model = input_model
    model.fit(X_train,y_train)
    preds = model.predict(X_valid)
    return np.sqrt(mean_squared_error(y_valid, preds))


input_model = LinearRegression()
print('LinearRegression:', score_dataset(X_train,X_test,y_train,y_test, input_model))

input_model = RandomForestRegressor()
print('RandomForestRegressor',score_dataset(X_train,X_test,y_train,y_test, input_model))

input_model = DecisionTreeRegressor()
print('DecisionTreeRegressor',score_dataset(X_train,X_test,y_train,y_test, input_model))

input_model = SVR()
print('SVR',score_dataset(X_train,X_test,y_train,y_test, input_model))

input_model = LinearSVR()
print('LinearSVR',score_dataset(X_train,X_test,y_train,y_test, input_model))

input_model = GradientBoostingRegressor()
print('GradientBoostingRegressor',score_dataset(X_train,X_test,y_train,y_test, input_model))

input_model = ElasticNet()
print('ElasticNet',score_dataset(X_train,X_test,y_train,y_test, input_model))


In [ ]:
# Si es necesario aplica ingeniería de variables

In [ ]:
from sklearn.model_selection import GridSearchCV
# Buscar los mejores hiperparámetros para el modelo
elastic_net_parag_grid =  {
                'alpha'     : [0.1,1,10,0.01,0.01,5],
                'l1_ratio'  :  np.arange(0.40,1.00,0.10),
            }

elastic_net_regressor = ElasticNet()

elastic_net_grid_search = GridSearchCV(
    elastic_net_regressor,
    param_grid=elastic_net_parag_grid,
    cv = 5,
    n_jobs=-1,
    verbose=1
)

elastic_net_grid_search.fit(X_train,y_train)
print('params:',elastic_net_grid_search.best_params_)

preds = elastic_net_grid_search.best_estimator_.predict(X_test)

final_test_score = np.sqrt(mean_squared_error(y_test, preds))

print('final_test_score:',final_test_score)

In [ ]:
# Entrena tu modelo con los mejores hiperparámetros
final_model=elastic_net_regressor = ElasticNet(alpha=1,l1_ratio=0.6)
final_model.fit(X_train,y_train)

In [ ]:
# Repite este proceso para otros modelos candidatos

### 5. Publica el modelo

In [ ]:
# Serializa el modelo
import pickle
pickle.dump(final_model, open('final_model.pkl','wb'))

In [ ]:
# Carga tu modelo en una aplicación
model = pickle.load(open('final_model.pkl','rb'))
model